The Goal of this notebook is to show how to use the rio-stac API to create a STAC Item from multiple assets.

In [ ]:
import datetime

import pystac
from pystac.utils import str_to_datetime

import rasterio

# Import extension version
from rio_stac.stac import PROJECTION_EXT_VERSION, RASTER_EXT_VERSION, EO_EXT_VERSION

# Import rio_stac methods
from rio_stac.stac import (
    get_dataset_geom,
    get_projection_info,
    get_raster_info,
    get_eobands_info,
    bbox_to_geom,
)

In [ ]:
assets = [
    {"name": "B01", "path": "./data/B01.tif", "href": None, "role": None},
    {"name": "B02", "path": "./data/B02.tif", "href": None, "role": None},
    {"name": "B03", "path": "./data/B03.tif", "href": None, "role": None},
    {"name": "B04", "path": "./data/B04.tif", "href": None, "role": None},
    {"name": "B05", "path": "./data/B05.tif", "href": None, "role": None},
]

media_type = pystac.MediaType.COG  # we could also use rio_stac.stac.get_media_type

# additional properties to add in the item
properties = {}

# datetime associated with the item
input_datetime = None

# STAC Item Id
id = "my_stac_item"

# name of collection the item belongs to
collection = None
collection_url = None

extensions =[
    f"https://stac-extensions.github.io/projection/{PROJECTION_EXT_VERSION}/schema.json",
    f"https://stac-extensions.github.io/raster/{RASTER_EXT_VERSION}/schema.json",
    f"https://stac-extensions.github.io/eo/{EO_EXT_VERSION}/schema.json",
]

In [ ]:
bboxes = []

pystac_assets = []

img_datetimes = []

for asset in assets:

    with rasterio.open(asset["path"]) as src_dst:

        # Get BBOX and Footprint
        dataset_geom = get_dataset_geom(src_dst, densify_pts=0, precision=-1)
        bboxes.append(dataset_geom["bbox"])

        if "start_datetime" not in properties and "end_datetime" not in properties:
            # Try to get datetime from https://gdal.org/user/raster_data_model.html#imagery-domain-remote-sensing
            dst_date = src_dst.get_tag_item("ACQUISITIONDATETIME", "IMAGERY")
            dst_datetime = str_to_datetime(dst_date) if dst_date else None
            if dst_datetime:
                img_datetimes.append(dst_datetime)

        proj_info = {
            f"proj:{name}": value
            for name, value in get_projection_info(src_dst).items()
        }

        raster_bands = get_raster_info(src_dst, max_size=1024)
        eo_bands = get_eobands_info(src_dst)

        bands = []
        if raster_bands or eo_bands:
            band_count = max(len(raster_bands), len(eo_bands))
            for idx in range(band_count):
                band = {}
                if idx < len(eo_bands):
                    band.update(eo_bands[idx])
                if idx < len(raster_bands):
                    band.update(raster_bands[idx])
                band.setdefault("name", f"b{idx + 1}")
                bands.append(band)

        cloudcover = src_dst.get_tag_item("CLOUDCOVER", "IMAGERY")
        if cloudcover is not None:
            properties.update({"eo:cloud_cover": int(cloudcover)})

        extra_fields = {**proj_info}
        if bands:
            extra_fields["bands"] = bands

        pystac_assets.append(
            (
                asset["name"],
                pystac.Asset(
                    href=asset["href"] or src_dst.name,
                    media_type=media_type,
                    extra_fields=extra_fields,
                    roles=asset["role"],
                ),
            )
        )

if img_datetimes and not input_datetime:
    input_datetime = img_datetimes[0]

input_datetime = input_datetime or datetime.datetime.utcnow()

minx, miny, maxx, maxy = zip(*bboxes)
bbox = [min(minx), min(miny), max(maxx), max(maxy)]

# item
item = pystac.Item(
    id=id,
    geometry=bbox_to_geom(bbox),
    bbox=bbox,
    collection=collection,
    stac_extensions=extensions,
    datetime=input_datetime,
    properties=properties,
)

# if we add a collection we MUST add a link
if collection:
    item.add_link(
        pystac.Link(
            pystac.RelType.COLLECTION,
            collection_url or collection,
            media_type=pystac.MediaType.JSON,
        )
    )

for key, asset in pystac_assets:
    item.add_asset(key=key, asset=asset)

item.validate()


In [ ]:
import json

print(json.dumps(item.to_dict(), indent=4))